# Hyperparameter tuning with Ray

Ray is a platform for distributed computing. It also has a module called tune which is the industry standard tool for tuning deep learning models by adjusting hyperparameters and architecturs.

In [ ]:
from torch import nn
import sys
sys.path.append('..')
from utils import get_dataloaders

train_loader, val_loader, test_loader = get_dataloaders()

class NN(nn.Module):
    def __init__(self, l1=120, l2=84):
        super().__init__()
        self.layers = torch.nn.Sequential([
            nn.Conv2d(1, 6, 5)
            nn.Conv2d(6, 16, 5)
            nn.Flatten()
            nn.Linear(16 * 5 * 5, l1)
            nn.Linear(l1, l2)
            nn.Linear(l2, 10)
        ])
       

    def forward(self, x):
        x = self.layers(x)
        return x

config = {
    'l1': 32,
    'l2': 16
}

nn = NN(**config)

config = {
    "l1": tune.sample_from(lambda _: 2 ** np.random.randint(2, 9)),
    "l2": tune.sample_from(lambda _: 2 ** np.random.randint(2, 9)),
    "lr": tune.loguniform(1e-4, 1e-1),
    "batch_size": tune.choice([2, 4, 8, 16])
}

scheduler = ASHAScheduler(
    metric="loss",
    mode="min",
    max_t=max_num_epochs,
    grace_period=1,
    reduction_factor=2
)

reporter = CLIReporter(
    # parameter_columns=["l1", "l2", "lr", "batch_size"],
    metric_columns=["loss", "accuracy", "training_iteration"]
)

result = tune.run(
    partial(train_cifar, data_dir=data_dir),
    resources_per_trial={"cpu": 2, "gpu": gpus_per_trial},
    config=config,
    num_samples=num_samples,
    scheduler=scheduler,
    progress_reporter=reporter
)